In [160]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd 
from cgra import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### KERNEL CONFIGURATION
Set up an application to be accelerated by loading the different kernel configurations into the kernel memory

In [97]:
# Load an existing kernel into memory
kmem_pos = 1
kmem_word = 0x802b

kmem = KER_CONF()
kmem.set_kernel_word(kmem_word, kmem_pos)
kmem.get_kernel_info(kmem_pos)

This kernel uses 43 instruction words starting at IMEM address 0.
It uses column(s): 0.
The SRF is located in SPM bank 0.


In [99]:
# Create a new kernel
kmem_pos = 2
kmem.set_kernel_params(num_instructions=20, imem_add_start=44, column_usage=3, srf_spm_addres=1, pos=kmem_pos)
print("Hex representation: " + kmem.get_word_in_hex(kmem_pos))
kmem.get_kernel_info(kmem_pos)

Hex representation: 0x38b14
This kernel uses 20 instruction words starting at IMEM address 44.
It uses column(s): both.
The SRF is located in SPM bank 1.


### Loop Control Unit IMEM 

In [163]:
# Load an existing imem word and decode it
imem_pos = 1
imem_word = 0x9c500

lcu_imem = LCU_IMEM()
lcu_imem.set_kernel_word(imem_word, imem_pos)
lcu_imem.get_instruction_info(imem_pos)

Immediate value: 0
LCU is in loop control mode
Performing ALU operation SSUB between operands SRF and ONE
Writing ALU result to LCU register 0


In [167]:
# Create a new kernel
imem_pos = 2
lcu_imem.set_kernel_params(imm=3, rf_wsel=2, rf_we=1, alu_op=1, br_mode=0, muxb_sel=7, muxa_sel=6, pos=imem_pos)
print("Hex representation: " + lcu_imem.get_word_in_hex(imem_pos))
lcu_imem.get_instruction_info(imem_pos)

Hex representation: 0xdc383
Immediate value: 3
LCU is in loop control mode
Performing ALU operation SADD between operands ZERO and ONE
Writing ALU result to LCU register 2
